<a href="https://colab.research.google.com/github/hwp28/hwp28/blob/main/%EB%94%A5%EB%9F%AC%EB%8B%9D_%EC%A2%85%EC%82%AC%EC%9E%90%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# 라이브러리 import 
# 패키지 및 라이브러리 import
import pandas as pd                       # dataframe
import numpy as np                        # array등 수치데이터 처리 
import matplotlib.pyplot as plt           # 그래프 

# 머신러닝 라이브러리 
from sklearn.model_selection import train_test_split

# 딥러닝 라이브러리 
from keras.utils import plot_model        # keras 모델 이미지 
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input

# keras 
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import LSTM

# 구글 드라이브 마운트 ( 자원 접근 권한 관련 )
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# 학습용 전체 데이터 import 
df = pd.read_csv('/content/drive/My Drive/project/2013_2019_분기별 종사자수.csv')
df.head()
df.columns

Index(['REG_DT', 'DIVIDE', 'Care practician', 'Electron practician ', 'TOTAL'], dtype='object')

In [5]:
# 전처리
# 전체 데이터(df_total: 전체종업자수)와 세부 데이터(df_care:보건) /(df_electron:전기운수업) 를 나눠서 학습 
df_total = df[['REG_DT','DIVIDE','TOTAL']]                        # 전체 취업자
df_care  = df[['REG_DT','DIVIDE','Care practician']]              # 보건업체 종사자
df_electron = df[['REG_DT','DIVIDE','Electron practician ']]      # 전기운수업 종사자
# df_electron = 
df_care = df_care.rename(columns={'Care practician':'CP'})                            # 컬럼명 길어서 변경 

In [6]:
df_electron.rename(columns={'Electron practician ':'EP'},inplace=True)  # 컬럼명 길어서 변경 

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
df_electron

In [ ]:
df_total = df_total.drop(['DIVIDE'], axis=1)
df_total

In [9]:
##############################################################################
# 함수 time split ( 연(year) 단위로 데이터 묶음)
##############################################################################
def split_xy(dataset, time_steps, y_column):
    x, y = list(), list()
    for i in range(len(dataset)):
        x_end_number = i + time_steps
        y_end_number = x_end_number + y_column # 수정

        if y_end_number > len(dataset):  # 수정
            break
        tmp_x = dataset[i:x_end_number, :]  # 수정
        tmp_y = dataset[x_end_number:y_end_number, 1]    # 수정
        x.append(tmp_x)
        y.append(tmp_y)
    return np.array(x), np.array(y)

In [10]:
x1, y1 = split_xy(df_total.values, 4, 1) # 4분기(1년치 데이터를 기준으로) 다음 1분기를 예측한다.

In [ ]:
x1

In [12]:
# 보건업 종사자 관련 데이터 Standard Scaling 
x2 = df_care[df_care['REG_DT'] >= 2014]
x2 = x2.drop(['REG_DT'], axis=1)
# df_care 데이터 전처리 
from sklearn.model_selection import train_test_split
x2_train, x2_test = train_test_split(x2, random_state=1,  test_size = 0.3)

#### 데이터 Scaling #####
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x2_train)
x2_train_scaled = scaler.transform(x2_train)
x2_test_scaled = scaler.transform(x2_test)

In [13]:
# 데이터 전처리 
from sklearn.model_selection import train_test_split
x1_train, x1_test, y1_train, y1_test = train_test_split(x1, y1, random_state=1,  test_size = 0.3)

# Standard Scaling 하기 위해 2차원 배열로 변경
x1_train = np.reshape(x1_train, (x1_train.shape[0], x1_train.shape[1] * x1_train.shape[2])) 
x1_test = np.reshape(x1_test, (x1_test.shape[0], x1_test.shape[1] * x1_test.shape[2]))

# #### 데이터 Scaling #####
from sklearn.preprocessing import StandardScaler
scaler1 = StandardScaler()
scaler1.fit(x1_train)
x1_train_scaled = scaler1.transform(x1_train)
x1_test_scaled = scaler1.transform(x1_test)
x1_train_scaled = np.reshape(x1_train_scaled, (x1_train_scaled.shape[0], 4, 2))
x1_test_scaled = np.reshape(x1_test_scaled, (x1_test_scaled.shape[0], 4, 2)) # 7일 단위 3개 feature 데이터를 저장 

In [14]:
# 데이터 전처리 
from sklearn.model_selection import train_test_split
x1_train, x1_test, y1_train, y1_test = train_test_split(x1, y1, random_state=1,  test_size = 0.3)

In [ ]:
# 모델 생성 및 학습 ( PV, RLORD_AMT 에 대한 lstm )
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, LSTM

study_size = 4 # 1년(4분기)
feature_size = 1 # 년도별 취업자수 feature 보겠다는 의미 

input1 = Input(shape=(study_size, 2))
dense1 = LSTM(64)(input1)
dense1 = Dense(32)(dense1)
dense1 = Dense(32)(dense1)
output1 = Dense(1)(dense1)
model1 = Model(inputs=[input1], outputs = output1)


# model1 = models.model_create_timeseries_for_concatenate(7, 3) # 7일단위 ( memory cell ) 2개 feature 데이터를 저장 
model1.compile(loss='mae', optimizer='adam')
model1.fit([x1_train_scaled], y1_train, validation_split=0.2, verbose=1, batch_size=2000, epochs=500)

In [16]:
# y1_pred = lstm 모델 (PV, RLORD_AMT, ORD_CNT) 로 예측한 RLORD_AMT 
loss = model1.evaluate([x1_train_scaled], y1_train, batch_size=1)
print('loss : ', loss)

y1_pred = model1.predict([x1_train_scaled])
print('<<<<<<<<<<model 1 예측>>>>>>>>>>')
for i in range(5):    
    print('(실제) : ', y1_test[i], '/ (예측) : ', y1_pred[i], '정확도(%) : ', y1_pred[i] / y1_test[i] * 100, '오차 : ',np.absolute(y1_test[i]-y1_pred[i]).astype(int))

19/19 [==============================] - 0s 2ms/step - loss: 341.5583
loss :  341.55828857421875
<<<<<<<<<<model 1 예측>>>>>>>>>>
(실제) :  [26749] / (예측) :  [26333.023] 정확도(%) :  [98.44488929] 오차 :  [415]
(실제) :  [27019] / (예측) :  [26132.896] 정확도(%) :  [96.72044296] 오차 :  [886]
(실제) :  [26971] / (예측) :  [26757.277] 정확도(%) :  [99.20758349] 오차 :  [213]
(실제) :  [26461] / (예측) :  [26589.457] 정확도(%) :  [100.48545796] 오차 :  [128]
(실제) :  [26998] / (예측) :  [26751.602] 정확도(%) :  [99.08734559] 오차 :  [246]


In [ ]:
# y1_pred 실제값과 비교한 그래프 
%matplotlib inline
import matplotlib.pylab as plt
plt.rcParams["figure.figsize"] = (14,10)
plt.rcParams['lines.linewidth'] = 2
plt.rcParams['lines.color'] = 'r'
plt.rcParams['axes.grid'] = True 

plt.plot(y1_train)
plt.plot(y1_pred)
plt.title('single model predict')
plt.legend(['real', 'test'])

In [18]:
# 비선형 학습 세트 추가 
df_care = df_care.drop(['DIVIDE'], axis=1)

In [19]:
# non linear model modeling 
dim = 2 # 입력 feature가 1개니까 
model2 = keras.Sequential()
model2.add(layers.Dense(10, input_dim=dim, activation='relu'))
model2.add(layers.Dense(5, activation='relu'))
model2.add(layers.Dense(1))  
model2.add(layers.Activation('relu'))

In [ ]:
# Keras Concatenate 
from tensorflow.keras.layers import concatenate
combinedInput = concatenate([model1.output, model2.output])
x = Dense(10, activation="relu")(combinedInput)
x = Dense(10, activation="relu")(x)
model_merged = Model(inputs=[model1.input, model2.input], outputs=x)
plot_model(model_merged, to_file='model_merged.png', show_shapes=True)

In [21]:
print(x1_train_scaled.shape)
print(x2_train_scaled.shape)

(19, 4, 2)
(19, 2)


In [ ]:
# fitting 
from tensorflow import keras
opt = keras.optimizers.Adam(learning_rate=0.1)
model_merged.compile(loss="mae", optimizer=opt)
hist = model_merged.fit(	
  [x1_train_scaled, x2_train_scaled], y1_train,  
  validation_split=0.2, 	
 	epochs=500,	batch_size=800)

In [ ]:
loss = model_merged.evaluate([x1_train_scaled, x2_train_scaled], y1_train, batch_size=1)
print('loss : ', loss)
y_pred = model_merged.predict([x1_train_scaled, x2_train_scaled])
print('<<<<<<<<<<merged model 예측>>>>>>>>>>')
for i in range(10):    
    print('(실제) : ', y1_train[i])
    print('(예측) : ', y1_pred[i].astype(int)) 
    print('오차           : ', np.absolute(y1_train[i] - y1_pred[i]).astype(int))    
    print('오차율(%)      : ', abs(y1_train[i]-y1_pred[i]) / y1_pred[i]*100)
    print()          

In [ ]:
# 1. LSTM + Non-linear(relu) 구성시 정확도 그래프 
plt.plot(y1_train, color='r')
plt.plot(y1_pred)
plt.legend(['real'], prop={'size': 22}, bbox_to_anchor=(0.99, 1.02))

In [25]:
# 2. LSTM + LSTM 구성시 정확도 
# x1, y1은 그대로 
# x2, y2는 lstm 
x2, y2 = split_xy(df_care.values, 4, 1) # 4분기(1년치 데이터를 기준으로) 다음 1분기를 예측한다.

# 데이터 전처리 
from sklearn.model_selection import train_test_split
x2_train, x2_test, y2_train, y2_test = train_test_split(x2, y2, random_state=1,  test_size = 0.3)

# Standard Scaling 하기 위해 2차원 배열로 변경
x2_train = np.reshape(x2_train, (x2_train.shape[0], x2_train.shape[1] * x2_train.shape[2])) 
x2_test = np.reshape(x2_test, (x2_test.shape[0], x2_test.shape[1] * x2_test.shape[2]))

# #### 데이터 Scaling #####
from sklearn.preprocessing import StandardScaler
scaler1 = StandardScaler()
scaler1.fit(x2_train)
x2_train_scaled = scaler1.transform(x2_train)
x2_test_scaled = scaler1.transform(x2_test)
x2_train_scaled = np.reshape(x2_train_scaled, (x2_train_scaled.shape[0], 4, 2))
x2_test_scaled = np.reshape(x2_test_scaled, (x2_test_scaled.shape[0], 4, 2)) # 7일 단위 3개 feature 데이터를 저장 

In [ ]:
# model2 remodeling ( model2는 nonlinear 였기 때문에 lstm으로 재구성)
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, LSTM

study_size = 4   # 1년(4분기)
feature_size = 1 # 년도별 취업자수 feature 보겠다는 의미 

input2 = Input(shape=(study_size, 2))
dense2 = LSTM(64)(input2)
dense2 = Dense(32)(dense2)
dense2 = Dense(32)(dense2)
output2 = Dense(1)(dense2)
model2 = Model(inputs=[input2], outputs = output2)

# Keras Concatenate 
from tensorflow.keras.layers import concatenate
combinedInput = concatenate([model1.output, model2.output])
x = Dense(10, activation="relu")(combinedInput)
x = Dense(10, activation="relu")(x)
model_merged = Model(inputs=[model1.input, model2.input], outputs=x)
plot_model(model_merged, to_file='model_merged.png', show_shapes=True)

In [ ]:
# fitting 
from tensorflow import keras
opt = keras.optimizers.Adam(learning_rate=0.1)
model_merged.compile(loss="mae", optimizer=opt)

hist = model_merged.fit(	
  [x1_train_scaled, x2_train_scaled], y2_train,  
  validation_split=0.2, 	
 	epochs=500,	batch_size=800)

In [ ]:
loss = model_merged.evaluate([x1_train_scaled, x2_train_scaled], y2_train, batch_size=1)
print('loss : ', loss)
y2_pred = model_merged.predict([x1_train_scaled, x2_train_scaled])
print('<<<<<<<<<<merged model 예측>>>>>>>>>>')
for i in range(10):    
    print('(실제) : ', y2_train[i])
    print('(예측) : ', y2_pred[i].astype(int))     
    print('오차           : ', np.absolute(y2_train[i] - y2_pred[i]).astype(int))    
    print('정확도(%)      : ', y2_pred[i] / y2_train[i] * 100)

In [34]:
# np.delete(y2_pred, np.where(y2_pred.astype(int)==0.0))
index = np.where(y2_pred == 0.).index

In [ ]:
# 1. LSTM + Non-linear(relu) 구성시 정확도 그래프 
plt.plot(y2_train, color='r')
plt.plot(y2_pred)
plt.ylim(2800, 3400)
plt.legend(['real'], prop={'size': 22}, bbox_to_anchor=(0.99, 1.02))